In [77]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def skew_matrix(v):
    """计算向量 v 的 skew矩阵"""
    return np.array([[0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]])

def construct_matrix_A(pts1_norm, pts2_norm):
    #八点法 构造A矩阵
    n = pts1_norm.shape[0] 
    A = np.zeros((n, 9))
    for i in range(n):
        x1, y1 = pts1_norm[i, :2]
        x2, y2 = pts2_norm[i, :2]
        A[i, :] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1]
    return A

def normalize_points(pts):
    #坐标归一化的矩阵表示
    n = pts.shape[0]
    mean = np.mean(pts, axis=0)#求平均值
    s = np.sqrt(2) / np.linalg.norm(pts-mean,axis=1).mean()
    T = np.array([[s, 0, -s * mean[0]],[0, s, -s * mean[1]],[0, 0, 1]])
    #计算T矩阵
    pts_norm = np.ones((n, 3))
    pts_norm[:, :3] = (pts - mean) * s#归一化坐标
    return pts_norm, T

def denormalize_matrix(F_norm, pts1, pts2):
    # 利用T解除归一化
    T2, T1 = normalize_points(pts2)[1], normalize_points(pts1)[1]
    F = np.dot(T2.T, np.dot(F_norm, T1))
    
    return F / F[2, 2]

def eight_point_algo_robust(pts1, pts2):
    #带鲁棒性的八点法求解F
    #利用归一化的点参与F计算
    pts1_norm, _ = normalize_points(pts1) 
    pts2_norm, _ = normalize_points(pts2)
    A = construct_matrix_A(pts1_norm, pts2_norm)
    _, _, V = np.linalg.svd(A)
    F = V[-1, :].reshape(3, 3)
    U, D, V = np.linalg.svd(F)
    D[-1] = 0
    F = np.dot(U, np.dot(np.diag(D), V))
    #计算归一化匹配点集的基础矩阵
    
    F = denormalize_matrix(F, pts1,pts2) 
    return F

def essential_from_fundamental(F, K):
    E = np.dot(K.T, np.dot(F, K))
    U, D, V = np.linalg.svd(E)
    D = np.diag([1, 1, 0])
    E = np.dot(U, np.dot(D, V))
    return E/E[2,2]

def ComputeRT(E):
    # 从本质矩阵恢复R和T
    U, _, VT = np.linalg.svd(E)

    if np.linalg.det(U) < 0:
        U[:, -1] *= -1
    if np.linalg.det(VT) < 0:
        VT[-1, :] *= -1

    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    WT = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]])

    R1 = np.dot(U, np.dot(W, VT))
    R2 = np.dot(U, np.dot(WT, VT))

    T1 = U[:, 2]
    T2 = -U[:, 2]

    return R1, R2, T1, T2

def fromFGetRT(points1,points2,K):
    #将F利用八点法分解dedaoRT
    F = eight_point_algo_robust(points1,points2)
    print(f'F is {F}\n')
    E_est = essential_from_fundamental(F, K)
    
    return ComputeRT(E)


In [100]:
def path2pairs(image1_path, image2_path):
    # 读取灰度图
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    gray1 = read_img(image1_path)
    gray2 = read_img(image2_path)
    # 程序参数读写无误
    # plt.imshow(img1)
    # plt.show()
    # 利用SIFT算子进行特征点识别
    kp1, des1 = extract_features(gray1)
    kp2, des2 = extract_features(gray2)
    # 输出两张图片的匹配结果
    # line2pics(img1, img2, kp1, des1, kp2, des2)
    # 进行特征点匹配，输出匹配数据
    data1, data2 = GetPairs(kp1, des1, kp2, des2)

    data1 = np.array(data1)
    data2 = np.array(data2)

    return data1, data2

def GetPairs(kp1, des1, kp2, des2):
    # 使用暴力搜索进行特征点匹配
    matches = match_keypoints(kp1, des1, kp2, des2)
    data1 = []
    data2 = []
    for m in matches:
        pt1 = [kp1[m[0]][0], kp1[m[0]][1]]
        pt2 = [kp2[m[1]][0], kp2[m[1]][1]]
        data1.append(pt1)
        data2.append(pt2)

    return data1, data2
def read_img(img_path):
    # 读取图片地址，转化为灰度图
    img = cv2.imread(img_path)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif len(img.shape) == 2:
        img = img
    img.astype('float32')
    return img


def extract_features(image):
    """
    提取图像特征
    """
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(image, None)
    # print(kp[0].pt[0],kp[0].pt[1],kp[0].size)
    kp2array = np.array([(k.pt[0], k.pt[1], k.size) for k in kp])

    # 检查是否修改了匹配关系
    return kp2array, des


def match_keypoints(kp1, des1, kp2, des2):
    matcher = cv2.BFMatcher(crossCheck=True)
    matches = matcher.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = np.array([(m.queryIdx, m.trainIdx) for m in matches])
    return matches

In [106]:


def four_point_homography(pts1, pts2):
    #四点法求解单应矩阵H
    pts1_norm, T1 = normalize_points(pts1)
    pts2_norm, T2 = normalize_points(pts2)
    #根据四点法原理将其拓展为8x9的矩阵
    A = np.zeros((2*pts1_norm.shape[0], 9))
    for i in range(pts1_norm.shape[0]):
        x, y = pts1_norm[i][0], pts1_norm[i][1]
        u, v = pts2_norm[i][0], pts2_norm[i][1]
        A[2 * i] = [-x, -y, -1, 0, 0, 0, x * u, y * u, u]
        A[2 * i + 1] = [0, 0, 0, -x, -y, -1, x * v, y * v, v]
        
    _, _, V = np.linalg.svd(A)
    H = V[-1].reshape((3, 3))
    # 解归一化
    H = np.dot(np.dot(np.linalg.inv(T2), H), T1)
    return H

def homography_calibrated(K, H_uncalib):
    #H矩阵的归一化
    H = np.linalg.inv(K) @ H_uncalib @ K
    #计算特征值 A^TA 和 AA^T 的非零特征值分别等于 $A$ 的奇异值的平方。
    eigvals = np.linalg.eigvals(H.T@H)
    H = H/np.sqrt(eigvals[1])
    return H

def get_motion_from_homography(H, K):
    #从单应矩阵H中取得变换矩阵
    H_calib = homography_calibrated(K, H)
    print(f'H is {H_calib}')
    HtH = H_calib.T @ H_calib
    eigvals, eigvecs = np.linalg.eig(HtH)#求解特征值和特征向量
    # print(eigvals)
    
    eigvals = eigvals[:]
    
    # print(eigvals)
    
    eigvecs = eigvecs[:, :]
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    u1 = (np.sqrt(1-eigvals[2])*eigvecs[:, 0]+np.sqrt(eigvals[0]-1)*eigvecs[:, 2])/np.sqrt(eigvals[0]-eigvals[2])
    u2 = (np.sqrt(1-eigvals[2])*eigvecs[:, 0]-np.sqrt(eigvals[0]-1)*eigvecs[:, 2])/np.sqrt(eigvals[0]-eigvals[2])
    U1 = np.stack([v2,u1,skew_matrix(v2)@u1],axis=1)
    W1 = np.stack([H_calib@v2,H_calib@u1,skew_matrix(H_calib@v2)@H_calib@u1],axis=1)
    U2 = np.stack([v2,u2,skew_matrix(v2)@u2],axis=1)
    W2 = np.stack([H_calib@v2,H_calib@u2,skew_matrix(H_calib@v2)@H_calib@u2],axis=1)
    R1 = W1@U1.T
    N1 = skew_matrix(v2)@u1
    T1 = (H_calib-R1)@N1
    R2 = W2@U2.T
    N2 = skew_matrix(v2)@u2
    T2 = (H_calib-R2)@N2
    R3 = R1
    N3 = -N1
    T3 = -T1
    R4 = R2
    N4 = -N2
    T4 = -T2
    return (R1,T1), (R2,T2), (R3,T3), (R4,T4)

def fromHGetE(pts1, pts2,K):
    H = four_point_homography(pts1, pts2)
    
    return get_motion_from_homography(H, K)



In [79]:
image1_path = 'pair2/000012.png' 
image2_path = 'pair2/000014.png'
intrinsics_path = 'pair2/K.txt'

data1,data2 = path2pairs(image1_path, image2_path)
'''设计读取相机内参K的函数'''
K = get_intrinsic_matrix(intrinsics_path)
data1,data2 = path2pairs(image1_path, image2_path)
data1h = np.concatenate((data1, np.ones((data1.shape[0], 1))), axis=-1)
data2h = np.concatenate((data2, np.ones((data2.shape[0], 1))), axis=-1)
# data1n = (np.linalg.inv(K) @ (data1h.T)).T
# data2n = (np.linalg.inv(K) @ (data2h.T)).T

In [85]:
'''八点法求解F矩阵，并将其转换为E矩阵分解成RT'''
R1,R2,R3,R4  =fromFGetRT(data1h,data2h,K)
print(R1)
print(R2)
print(R3)
print(R4)

F is [[ 6.55438838e-07  2.38560206e-06 -2.40215392e-03]
 [-2.53897532e-06  9.56561289e-07 -8.45397218e-04]
 [ 1.95072881e-03 -1.08703324e-03  1.00000000e+00]]

[[ 9.69991506e-01 -2.43136538e-01  1.04945410e-03]
 [-2.43137831e-01 -9.69990797e-01  1.35972142e-03]
 [ 6.87362861e-04 -1.57408022e-03 -9.99998525e-01]]
[[-0.49865119 -0.70547199  0.50364299]
 [ 0.7951795  -0.14103538  0.58974451]
 [-0.34501675  0.69456338  0.63130433]]
[-0.58667161 -0.6871196   0.42858264]
[ 0.58667161  0.6871196  -0.42858264]


In [107]:
(R1,T1), (R2,T2), (R3,T3), (R4,T4) = fromHGetE(data1h,data2h,K)
print()
print(R1)
print(T1)
print()
print(R2)
print(T2)
print()
print(R3)
print(T3)
print()
print(R4)
print(T4)

H is [[-0.56069919  0.24762068  0.02279639]
 [-0.55287516 -0.8640333  -0.07272635]
 [ 1.0444677  -1.38544761 -0.47771764]]

[[-0.74921183  0.54245862  0.38002669]
 [-0.58830551 -0.80861946 -0.00558606]
 [ 0.30426677 -0.22775694  0.92495865]]
[-0.50007991 -0.09398842 -1.96357986]

[[ 0.16762793 -0.88376436  0.4368769 ]
 [-0.9101729  -0.30900617 -0.27586314]
 [ 0.37879567 -0.35139115 -0.85617644]]
[-1.4078193   0.69063837  1.28671017]

[[-0.74921183  0.54245862  0.38002669]
 [-0.58830551 -0.80861946 -0.00558606]
 [ 0.30426677 -0.22775694  0.92495865]]
[0.50007991 0.09398842 1.96357986]

[[ 0.16762793 -0.88376436  0.4368769 ]
 [-0.9101729  -0.30900617 -0.27586314]
 [ 0.37879567 -0.35139115 -0.85617644]]
[ 1.4078193  -0.69063837 -1.28671017]


**设计读取相机内参K的函数**